In [7]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex
import requests
import json
import time
import gzip
import io
import csv

In [ ]:
https://gz.blockchair.com/bitcoin/blocks/blockchair_bitcoin_blocks_20180101.tsv.gz

In [2]:
url = 'https://gz.blockchair.com/bitcoin/blocks/blockchair_bitcoin_blocks_20180101.tsv.gz'
response = requests.get(url)

In [8]:
compressed_data = io.BytesIO(response.content)
with gzip.GzipFile(fileobj=compressed_data) as f:
    decompressed_data = f.read()

# Step 3: Convert the TSV content to a dictionary
# Using csv.DictReader to parse the TSV data
tsv_data = io.StringIO(decompressed_data.decode('utf-8'))
reader = csv.DictReader(tsv_data, delimiter='\t')

# Convert to a list of dictionaries
data_list = [row for row in reader]

In [11]:
for data in data_list:
    id_       = data['id']
    timestamp = data['time']
    fee       = data['fee_total']
    blockrew  = data['generation']
    print((id_, timestamp, fee, blockrew))

('501961', '2018-01-01 00:08:28', '475069391', '1250000000')
('501962', '2018-01-01 00:25:00', '386453524', '1250000000')
('501963', '2018-01-01 00:29:00', '135761522', '1250000000')
('501964', '2018-01-01 00:33:42', '206283332', '1250000000')
('501965', '2018-01-01 00:41:38', '263044095', '1250000000')
('501966', '2018-01-01 00:44:05', '117864387', '1250000000')
('501967', '2018-01-01 00:47:58', '174321692', '1250000000')
('501968', '2018-01-01 01:02:03', '408714195', '1250000000')
('501969', '2018-01-01 01:20:48', '449943300', '1250000000')
('501970', '2018-01-01 01:21:28', '62146018', '1250000000')
('501971', '2018-01-01 01:44:42', '569103967', '1250000000')
('501972', '2018-01-01 01:55:07', '273046566', '1250000000')
('501973', '2018-01-01 01:56:34', '109336305', '1250000000')
('501974', '2018-01-01 02:10:45', '402760789', '1250000000')
('501975', '2018-01-01 02:11:31', '70462819', '1250000000')
('501976', '2018-01-01 02:29:54', '484332387', '1250000000')
('501977', '2018-01-01 02:

In [12]:
from datetime import datetime, timedelta

def generate_dates(start_date, end_year):
    dates = []
    current_date = start_date
    
    while current_date.year >= end_year:
        dates.append(current_date.strftime('%Y%m%d'))
        current_date -= timedelta(days=1)
    
    return dates

In [16]:
start_date = datetime.now() - timedelta(days=1)
    
# End year (e.g., 2020)
end_year = 2020

# Generate dates
date_list = generate_dates(start_date, end_year)
date_list[0]

'20240902'

In [22]:
def getDateData(date):
    url = 'https://gz.blockchair.com/bitcoin/blocks/blockchair_bitcoin_blocks_{}.tsv.gz'.format(date)
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    with gzip.GzipFile(fileobj=compressed_data) as f:
        decompressed_data = f.read()
    tsv_data = io.StringIO(decompressed_data.decode('utf-8'))
    reader = csv.DictReader(tsv_data, delimiter='\t')
    data_list = [row for row in reader]
    out = []
    for data in data_list:
        id_       = data['id']
        timestamp = data['time']
        fee       = data['fee_total']
        blockrew  = data['generation']
        out.append((id_, timestamp, fee, blockrew))
    return out

In [26]:
fullOut = []
for date in tqdm(date_list):
    out = getDateData(date)
    fullOut.extend(out)
    time.sleep(1)

100%|██████████| 1707/1707 [1:03:31<00:00,  2.23s/it]


In [ ]:
fullOut

In [27]:
len(fullOut)

248919

In [30]:
sortedOut = sorted(fullOut, key=lambda x: x[0])

In [35]:
sortedOut[0]

('610691', '2020-01-01 00:03:05', '17534026', '1250000000')

In [36]:
for i in range(len(sortedOut)):
    row = sortedOut[i]
    if int(row[0]) == 665399:
        print(i)
        break

54708


In [37]:
sortedOut[54708]

('665399', '2021-01-10 10:12:49', '61779496', '625000000')

In [40]:
file_name = 'height_time_fees_blockrew.csv'

# Writing the list of tuples to a CSV file
with open(file_name, mode='a', newline='') as file:
    writer = csv.writer(file)
    
    # Write each tuple as a row in the CSV file
    for row in sortedOut[54709:]:
        writer.writerow(row)